<a href="https://colab.research.google.com/github/pawarharshit/tensorflow_programs/blob/master/Dance_forms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls "drive/My Drive"

'Colab Notebooks'	     CS001_KE.twb    'Getting started.pdf'
'Copy of facenet_keras.h5'   dance_form.zip  'MY Family'


In [2]:
! mkdir dance_form

mkdir: cannot create directory ‘dance_form’: File exists


In [6]:
!unzip "drive/My Drive/dance_form.zip" -d dance_form

Archive:  drive/My Drive/dance_form.zip
   creating: dance_form/dataset/
   creating: dance_form/dataset/test/
  inflating: dance_form/dataset/test/508.jpg  
  inflating: dance_form/dataset/test/246.jpg  
  inflating: dance_form/dataset/test/473.jpg  
  inflating: dance_form/dataset/test/485.jpg  
  inflating: dance_form/dataset/test/128.jpg  
  inflating: dance_form/dataset/test/410.jpg  
  inflating: dance_form/dataset/test/465.jpg  
  inflating: dance_form/dataset/test/196.jpg  
  inflating: dance_form/dataset/test/340.jpg  
  inflating: dance_form/dataset/test/467.jpg  
  inflating: dance_form/dataset/test/306.jpg  
  inflating: dance_form/dataset/test/171.jpg  
  inflating: dance_form/dataset/test/323.jpg  
  inflating: dance_form/dataset/test/332.jpg  
  inflating: dance_form/dataset/test/330.jpg  
  inflating: dance_form/dataset/test/170.jpg  
  inflating: dance_form/dataset/test/479.jpg  
  inflating: dance_form/dataset/test/9.jpg  
  inflating: dance_form/dataset/test/283.jpg 

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
import os
import tensorflow as tf

In [4]:
train = pd.read_csv('dance_form/dataset/train.csv')

In [5]:
train_images = r'dance_form/dataset/train'

In [6]:
train_image = list()
for i in tqdm(range(train.shape[0])):
    image_name = train.iloc[i].Image
    image_path = os.path.join(train_images,image_name)
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(150,150,3))
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = image_array/255.0
    train_image.append(image_array)
    
X = np.array(train_image)

100%|██████████| 364/364 [00:02<00:00, 151.52it/s]


In [7]:
X.shape

(364, 150, 150, 3)

In [8]:
y = train['target']

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [10]:
forms = encoder.classes_

In [11]:
X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=42,test_size=0.3)

In [12]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
# datagen_train = ImageDataGenerator(
#     rescale=1./255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)


# datagen_val = ImageDataGenerator(
#     rescale=1./255,
#     featurewise_center=True,
#     featurewise_std_normalization=True)

In [14]:
# train_data = datagen_train.flow(X_train, y_train, batch_size=32)
# val_data =  datagen_train.flow(X_val, y_val, batch_size=32)

In [15]:
# train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
train_data = tf.data.Dataset.from_tensor_slices((X, y))


In [16]:
train_data

<TensorSliceDataset shapes: ((150, 150, 3), ()), types: (tf.float32, tf.int64)>

In [17]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_data = train_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# val_data = val_data.batch(BATCH_SIZE)

In [18]:
model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32,padding="same",strides=2, kernel_size=(3, 3),activation='relu',input_shape=(150,150,3)),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            tf.keras.layers.Conv2D(64,padding="same",strides=2,kernel_size=(2,2),activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            # tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256,activation='relu'),
            # tf.keras.layers.Dense(256,activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(8,activation='softmax')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 75, 75, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 64)        8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1327360   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0

In [20]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [21]:
history = model.fit(train_data,epochs=30)

Epoch 1/30
6/6 [==============================] - 1s 191ms/step - loss: 2.0782 - accuracy: 0.1511
Epoch 2/30
6/6 [==============================] - 1s 190ms/step - loss: 2.0309 - accuracy: 0.1951
Epoch 3/30
6/6 [==============================] - 1s 202ms/step - loss: 1.9771 - accuracy: 0.3462
Epoch 4/30
6/6 [==============================] - 1s 191ms/step - loss: 1.9107 - accuracy: 0.4121
Epoch 5/30
6/6 [==============================] - 1s 192ms/step - loss: 1.8652 - accuracy: 0.4341
Epoch 6/30
6/6 [==============================] - 1s 192ms/step - loss: 1.8324 - accuracy: 0.4780
Epoch 7/30
6/6 [==============================] - 1s 191ms/step - loss: 1.8207 - accuracy: 0.4643
Epoch 8/30
6/6 [==============================] - 1s 195ms/step - loss: 1.7631 - accuracy: 0.5467
Epoch 9/30
6/6 [==============================] - 1s 192ms/step - loss: 1.7323 - accuracy: 0.5632
Epoch 10/30
6/6 [==============================] - 1s 194ms/step - loss: 1.6913 - accuracy: 0.6099
Epoch 11/30
6/6 [==

In [22]:
test = pd.read_csv('dance_form/dataset/test.csv')

In [23]:
test_images = r'dance_form/dataset/test'

In [25]:
test_image = list()
for i in tqdm(range(test.shape[0])):
    image_name = test.iloc[i].Image
    image_path = os.path.join(test_images,image_name)
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(150,150,3))
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = image_array/255.0
    test_image.append(image_array)
    
X_test = np.array(test_image)

100%|██████████| 156/156 [00:01<00:00, 142.56it/s]


In [30]:
y_test = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [32]:
y_test_label = encoder.inverse_transform(y_test)

In [34]:
y_test_label

156

In [36]:
test['target'] = y_test_label

In [38]:
test.to_csv('prediction.csv')